<a href="https://colab.research.google.com/github/hun9008/ML_TeamProject_24SS/blob/main/PreTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 400장으로 학습한 결과 일반적으로 CNN, ResNet 모두
## train error는 낮으나, test error가 높음. Overfitting.
## 같은 조건에서 CNN < ResNet

---



In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
!pip install -q keras

In [4]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt

In [5]:
import os
os.chdir("/content/drive/My Drive/ML_TeamProject/preprocessed_images_40")
!ls

mild  normal  severe  TEST_DIR	TRAIN_DIR


In [37]:
train = 'TRAIN_DIR/'
test = 'TEST_DIR/'

# Augmentation with only zoom and grayscale format
# Set validation_split to split a portion of training data into validation data
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip = True,
                                   zoom_range = 0.3,
                                   validation_split=0.2)  # 20% of the data will be used for validation

# Corresponding generator for training data
train_generator = train_datagen.flow_from_directory(
    train,
    color_mode='grayscale',
    batch_size=32,
    class_mode = 'categorical',
    target_size=(224,224),
    subset='training',  # Use 'training' subset
    seed=42)

# Corresponding generator for validation data
validation_generator = train_datagen.flow_from_directory(
    train,
    color_mode='grayscale',
    batch_size=32,
    class_mode = 'categorical',
    target_size=(224,224),
    subset='validation',  # Use 'validation' subset
    seed=42 )

# ImageDataGenerator for testing data
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    test,
    color_mode='grayscale',
    batch_size=32,
    class_mode = 'categorical',
    target_size=(224,224),
    seed=42 )

x_train, y_train = next(train_generator)
x_validation, y_validation = next(validation_generator)
x_test, y_test = next(test_generator)

print(x_train.shape)
print(x_validation.shape)
print(x_test.shape)

Found 329 images belonging to 3 classes.
Found 81 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 224, 224, 1)
(32, 224, 224, 1)
(32, 224, 224, 1)


# Basic CNN

In [38]:
#Basic CNN Model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
import numpy as np
from keras.callbacks import ModelCheckpoint

In [39]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),padding='same', activation='relu', input_shape = (224, 224, 1)))
model.add(MaxPool2D(pool_size=(1,1)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [40]:
#for fitting the model
from keras.callbacks import EarlyStopping

stop = EarlyStopping(
    monitor = 'val_accuracy',
    mode='max',
    patience= 8,
    verbose = 2,
    restore_best_weights = True
)

STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size
EPOCHS = 20

results = model.fit(
train_generator,
steps_per_epoch=STEPS,
epochs=EPOCHS,
validation_data=validation_generator,
validation_steps=VAL_STEPS,
callbacks=[stop]
)

Epoch 1/20
10/10 [==============================] - 5s 423ms/step - loss: 17014.5215 - accuracy: 0.3064 - val_loss: 5328.7842 - val_accuracy: 0.3594
Epoch 2/20
10/10 [==============================] - 3s 292ms/step - loss: 2355.6443 - accuracy: 0.3737 - val_loss: 1451.3693 - val_accuracy: 0.4375
Epoch 3/20
10/10 [==============================] - 3s 297ms/step - loss: 846.6544 - accuracy: 0.5421 - val_loss: 479.3438 - val_accuracy: 0.5781
Epoch 4/20
10/10 [==============================] - 2s 208ms/step - loss: 294.3356 - accuracy: 0.6667 - val_loss: 364.8170 - val_accuracy: 0.4844
Epoch 5/20
10/10 [==============================] - 2s 212ms/step - loss: 168.8932 - accuracy: 0.7104 - val_loss: 340.2712 - val_accuracy: 0.5156
Epoch 6/20
10/10 [==============================] - 2s 208ms/step - loss: 118.0137 - accuracy: 0.7677 - val_loss: 308.8977 - val_accuracy: 0.5312
Epoch 7/20
10/10 [==============================] - 2s 211ms/step - loss: 108.1772 - accuracy: 0.7542 - val_loss: 334.4

# ResNet50

In [41]:
from keras import applications
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import SGD, Adam

In [42]:
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (224,224,1))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation= 'softmax')(x)
resnetmodel = Model(inputs = base_model.input, outputs = predictions)
adam = Adam(learning_rate=0.001)
resnetmodel.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
EPOCH = 30
stopping_resNet = EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5,
    verbose = 2,
    restore_best_weights = True
)

restNetModel = resnetmodel.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCH,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[stopping_resNet]
)

Epoch 1/30
10/10 [==============================] - 33s 805ms/step - loss: 2.4783 - accuracy: 0.4478 - val_loss: 209.9615 - val_accuracy: 0.2812
Epoch 2/30
10/10 [==============================] - 4s 380ms/step - loss: 1.6496 - accuracy: 0.6061 - val_loss: 408.0794 - val_accuracy: 0.3906
Epoch 3/30
10/10 [==============================] - 4s 409ms/step - loss: 0.9174 - accuracy: 0.6970 - val_loss: 410.6113 - val_accuracy: 0.4062
Epoch 4/30
10/10 [==============================] - 4s 373ms/step - loss: 0.6088 - accuracy: 0.8316 - val_loss: 110.8672 - val_accuracy: 0.2344
Epoch 5/30
10/10 [==============================] - 4s 407ms/step - loss: 0.5949 - accuracy: 0.8316 - val_loss: 139.3368 - val_accuracy: 0.2500
Epoch 6/30
10/10 [==============================] - 4s 398ms/step - loss: 0.5529 - accuracy: 0.8485 - val_loss: 16.4466 - val_accuracy: 0.2969
Epoch 7/30
10/10 [==============================] - 4s 370ms/step - loss: 0.4296 - accuracy: 0.8653 - val_loss: 4.9412 - val_accuracy: 0

In [44]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 32)      320       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 224, 224, 32)      0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 1605632)           0         
                                                                 
 dense_8 (Dense)             (None, 100)               160563300 
                                                                 
 dense_9 (Dense)             (None, 3)                 303       
                                                                 
Total params: 160563923 (612.50 MB)
Trainable params: 160563923 (612.50 MB)
Non-trainable params: 0 (0.00 Byte)
________

In [45]:
resnetmodel.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 1)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         3200      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [46]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(32, 224, 224, 1)
(32, 224, 224, 1)
(32, 3)
(32, 3)


In [47]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [51]:
predictions = model.predict(x_test)
predictions_onehot = np.zeros_like(predictions)
predictions_onehot[np.arange(len(predictions)), predictions.argmax(1)] = 1

print("test label = \n", y_test)
print("prediction = \n", predictions_onehot)

1/1 [==============================] - 0s 22ms/step
test label = 
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
prediction = 
 [[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [52]:
pred_size = len(predictions_onehot)
correct = 0

for i in range(len(predictions_onehot)):
    for j in range(3):
        if predictions_onehot[i][j] == y_test[i][j]:
            correct += 1
        else :
            print("wrong")
            break

print("accuracy = ", correct/pred_size)

wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
accuracy =  1.3125


In [53]:
predictions = resnetmodel.predict(x_test)
predictions_onehot = np.zeros_like(predictions)
predictions_onehot[np.arange(len(predictions)), predictions.argmax(1)] = 1

print("test label = \n", y_test)
print("prediction = \n", predictions_onehot)

1/1 [==============================] - 0s 27ms/step
test label = 
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
prediction = 
 [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [54]:
pred_size = len(predictions_onehot)
correct = 0

for i in range(len(predictions_onehot)):
    for j in range(3):
        if predictions_onehot[i][j] == y_test[i][j]:
            correct += 1
        else :
            print("wrong")
            break

print("accuracy = ", correct/pred_size)

wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
wrong
accuracy =  1.03125
